In [20]:
import mysql.connector
import json

# Establish connection to the MySQL database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Edu-helplimited001',
    database='sms'
)

cursor = connection.cursor()

# Fetch student data with the JSON column
query = "SELECT student_id, subject_scores FROM academic_records"
cursor.execute(query)
rows = cursor.fetchall()

# Close the cursor and connection after fetching
cursor.close()
connection.close()


In [21]:
# Function to calculate average score for each subject from JSON data
def calculate_subject_averages(subject_scores_json):
    # Parse the JSON string
    subject_scores = json.loads(subject_scores_json)
    
    # Dictionary to store total scores and counts for each subject
    subject_totals = {}
    
    # Iterate over terms and subjects to calculate the total score for each subject
    for term, subjects in subject_scores.items():
        for subject, score in subjects.items():
            if subject not in subject_totals:
                subject_totals[subject] = {'total_score': 0, 'term_count': 0}
            subject_totals[subject]['total_score'] += score
            subject_totals[subject]['term_count'] += 1
    
    # Calculate the average for each subject
    subject_averages = {}
    for subject, data in subject_totals.items():
        average_score = data['total_score'] / data['term_count'] if data['term_count'] > 0 else 0
        subject_averages[subject] = average_score
    
    return subject_averages

# Create a list to store the results in the format (student_id, subject, average_score)
subject_averages_list = []

# Loop through each row and calculate the average score for each subject
for row in rows:
    student_id = row[0]
    subject_scores_json = row[1]
    subject_averages = calculate_subject_averages(subject_scores_json)
    
    # Append each subject's average score to the list
    for subject, avg_score in subject_averages.items():
        subject_averages_list.append((student_id, subject, avg_score))


In [22]:
subject_averages_list

[(1, 'Yoruba', 71.25),
 (1, 'Biology', 70.375),
 (1, 'English', 53.875),
 (1, 'Physics', 66.0),
 (1, 'Chemistry', 53.5),
 (1, 'Mathematics', 51.625),
 (1, 'Civic Education', 51.875),
 (1, 'Data Processing', 60.75),
 (1, 'Computer Science', 68.0),
 (1, 'Agricultural Science', 55.125),
 (2, 'English', 55.25),
 (2, 'Physics', 48.75),
 (2, 'Chemistry', 62.0),
 (2, 'Economics', 51.375),
 (2, 'Fisheries', 60.0),
 (2, 'Visual Art', 54.75),
 (2, 'Mathematics', 55.5),
 (2, 'Data Processing', 61.875),
 (2, 'Agricultural Science', 53.125),
 (3, 'Yoruba', 53.375),
 (3, 'English', 65.375),
 (3, 'Physics', 53.5),
 (3, 'Chemistry', 58.625),
 (3, 'Mathematics', 57.75),
 (3, 'Civic Education', 63.0),
 (3, 'Data Processing', 62.75),
 (3, 'Computer Science', 52.625),
 (3, 'Further Mathematics', 63.0),
 (3, 'Agricultural Science', 56.0),
 (4, 'English', 54.0),
 (4, 'Physics', 53.875),
 (4, 'Chemistry', 45.5),
 (4, 'Visual Art', 49.5),
 (4, 'Mathematics', 48.625),
 (4, 'Data Processing', 56.625),
 (4, 'Com

In [24]:
# Dynamically add columns for each subject in academic_records table
subjects = set()
for student_id, subject, avg_score in subject_averages_list:
    subjects.add(subject)

# Reconnect to the database
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Edu-helplimited001',
    database='sms'
)
cursor = connection.cursor()

# Alter the table to add a new column for each subject
for subject in subjects:
    alter_query = f"ALTER TABLE academic_records ADD COLUMN {subject.replace(' ', '_')}_avg FLOAT"
    cursor.execute(alter_query)

# Commit the changes
connection.commit()


In [25]:
# Update each student's average score for each subject
for student_id, subject, avg_score in subject_averages_list:
    update_query = f"UPDATE academic_records SET {subject.replace(' ', '_')}_avg = %s WHERE student_id = %s"
    cursor.execute(update_query, (avg_score, student_id))

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()
